In [7]:
import pandas as pd

In [24]:
df=pd.read_csv("IIT GUWAHATI/training_set_labels.csv")

In [26]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['respondent_id']=le.fit_transform(df['respondent_id'])
df

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop("seasonal_vaccine",axis='columns'),df.seasonal_vaccine)

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
model=LogisticRegression()

In [30]:
model.fit(x_train,y_train)

LogisticRegression()

In [31]:
from sklearn import linear_model

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   respondent_id     26707 non-null  int64
 1   xyz_vaccine       26707 non-null  int64
 2   seasonal_vaccine  26707 non-null  int64
dtypes: int64(3)
memory usage: 626.1 KB


In [33]:
model=linear_model.LinearRegression()

In [34]:
model

LinearRegression()

In [35]:
x=df.seasonal_vaccine

In [36]:
y=model.predict(x)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [16]:
import jax.numpy as np
from jax import grad,vmap,jit,pmap

# deeplearning in numpy

In [17]:
def predict(params,inputs):
    for w,b in params:
        outputs=np.dot(inputs,w)+b
        inputs=np.tanh(outputs)
        return outputs
    
def mse_loss(params,batch):
    inputs,targets=batch
    preds=predict(params,inputs)
    return np.sum(preds-targets**2)

In [18]:
gradient_fun=jit(grad(mse_loss))

In [19]:
perexample_grads=jit(vmap(grad(mse_loss),in_axes=(None,0)))

In [20]:
parallel_grads=jit(pmap(grad(mse_loss),in_axes=(None,0)))

In [24]:
gradient_fun,perexample_grads,parallel_grads

(<PjitFunction of <function mse_loss at 0x0000020C14E95A80>>,
 <PjitFunction of <function mse_loss at 0x0000020C14E95760>>,
 <PjitFunction of <PmapFunction object at 0x0000020C14E82C80>>)

In [25]:
import jax.numpy as np

In [26]:
from jax import random

In [29]:
layer_sizes=[784,512,512,10]
stop_size=0.01
num_epochs=10
batch_size=128
n_targets=10

In [33]:
def random_layer_params(m,n,key,scale=1e-2):
  w_key,b_key=random.split(key)
  return scale *random_normal(w_key,(n,m)),scale*random.normal(b_key,(n,))

In [41]:
def init_network_params(sizes,key):
  keys=random.split(key,len(sizes))
  return [random_layer_params(m,n,k)for m, n, k in zip(sizes[:-1],sizes[1:],keys)]


In [42]:
params=init_network_params(layer_sizes,random.key(0))

NameError: name 'random_normal' is not defined

In [46]:
# A helper function to randomty initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
 w_key, b_key = random.split(key)
 return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
 keys = random. split(key, len(sizes))
 return [random_layer_params(m, n, k) for m, n, k in zip(sizes[ :- 1], sizes[1:], keys)]

params = init_network_params(layer_sizes, random.key(0))

In [51]:
from jax.scipy.special import logsumexp

def relu(x):
 return jnp.maximum(0, x)

def predict(params, image):
# per-example predictions
 activations = image
 for w, b in params[ :- 1] :
   outputs = jnp.dot(w, activations) + b
   activations = relu(outputs)

 final_w, final_b = params[-1]
 logits = jnp.dot(final_w, activations) + final_b
 return logits - logsumexp(logits)

In [56]:
# This works on single examples
random_flattened_image = random.normal(random.key(1), (10,28 *28,))
try:
 preds = predict(params, random_flattened_images)
except TypeError:
    print('Invalid shapes')
print (preds. shape)



# Output:
# (10,)

# Let's upgrade it to handle batches using 'vmap

# Make a batched version of the 'predict' function
batched_predict = vmap(predict, in_axes=(None, 0))

# 'batched_predict' has the same call signature as 'predict'
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

# Output:
(10, 10)

NameError: name 'random_flattened_images' is not defined

In [58]:
def one_hot(x, k, dtype=jnp.float32):
 """Create a one-hot encoding of x of size k."""
 return jnp.array(x[:, None] == jnp.arange(k), dtype)

def accuracy(params, images, targets):
 target_class = jnp.argmax(targets, axis=1)
 predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
 return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
 preds = batched_predict(params, images)
 return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
 grads = grad(loss)(params, x, y)
 return [(w - step_size * dw, b - step_size * db)
 for (w, b), (dw, db) in zip(params, grads)]


NameError: name 'jnp' is not defined

In [64]:
def get_train_batches():
# as_supervised=True gives us the (image, label) as a tuple instead of a dict
 ds = tfds.load(name='mnist', split='train', as_supervised=True, data_dir=data_dir)
# You can build up an arbitrary tf.data input pipeline
 ds = ds.batch(batch_size).prefetch(1)
# tfds.dataset_as_numpy converts the tf.data. Dataset into an iterable of NumPy arrays
 return tfds.as_numpy(ds)

for epoch in range(num_epochs):
 for x, y in get_train_batches( ):
  x = jnp.reshape(x, (len(x), num_pixels))
  y = one_hot(y, num_labels)
  params = update(params, x, y)

train_acc = accuracy(params, train_images, train_labels)
test_acc = accuracy(params, test_images, test_labels)
print("Training set accuracy {}". format(train_acc))
print( "Test set accuracy {}". format(test_acc))

NameError: name 'tfds' is not defined

In [76]:
import jax.numpy as jnp
from jax import random, grad, jit, vmap
from jax.scipy.special import logsumexp
import tensorflow_datasets as tfds

# Define the layer sizes and other constants
layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 10
batch_size = 128

# Helper functions for initializing and processing the neural network
def random_layer_params(m, n, key, scale=1e-2):
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

def init_network_params(sizes, key):
    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

params = init_network_params(layer_sizes, random.PRNGKey(0))

# Activation function
def relu(x):
    return jnp.maximum(0, x)

# Define the predict function
def predict(params, image):
    activations = image
    for w, b in params[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
    
    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b
    return logits - logsumexp(logits)

# Batched version of the predict function
batched_predict = vmap(predict, in_axes=(None, 0))

# One-hot encoding function
def one_hot(x, k, dtype=jnp.float32):
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

# Accuracy function
def accuracy(params, images, targets):
    target_class = jnp.argmax(targets, axis=1)
    predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
    return jnp.mean(predicted_class == target_class)

# Loss function
def loss(params, images, targets):
    preds = batched_predict(params, images)
    return -jnp.mean(preds * targets)

# Compiled update function with JIT
@jit
def update(params, x, y):
    grads = grad(loss)(params, x, y)
    return [(w - step_size * dw, b - step_size * db) for (w, b), (dw, db) in zip(params, grads)]

# Function to get training batches using TensorFlow Datasets
def get_train_batches():
    ds = tfds.load(name='mnist', split='train', as_supervised=True)
    ds = ds.batch(batch_size).prefetch(1)
    return tfds.as_numpy(ds)

# Training loop
for epoch in range(num_epochs):
    for x, y in get_train_batches():
        x = jnp.reshape(x, (len(x), 784))  # Assuming 784 pixels for MNIST images
        y = one_hot(y, 10)  # 10 classes for MNIST
        
        params = update(params, x, y)

# Evaluate accuracy on training and test sets
train_acc = accuracy(params, train_images, train_labels)  # Provide train_images and train_labels
test_acc = accuracy(params, test_images, test_labels)  # Provide test_images and test_labels
print("Training set accuracy: {}".format(train_acc))
print("Test set accuracy: {}".format(test_acc))


NameError: name 'train_images' is not defined